In [3]:
print("ok")

ok


In [4]:
%pwd

'c:\\Users\\lokes\\Medichat\\research'

In [5]:
import os
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\lokes\\Medichat'

In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    
    documents=loader.load()
    
    return documents

In [9]:
extracted_data=load_pdf_file(data='Data/')

In [10]:
# extracted_data

In [11]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [13]:
# text_chunks

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [16]:
embeddings=download_hugging_face_embeddings()

C:\Users\lokes\AppData\Local\Temp\ipykernel_22492\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
#query_result

In [38]:
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [21]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "mbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [40]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [41]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [42]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [43]:
docsearch

In [44]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [45]:
retrieved_docs = retriever.invoke("What is Acne?")

In [46]:
retrieved_docs

[Document(id='ab9ebf4d-7169-407b-a2e5-8a3ef338e061', metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='e1dd86a6-18de-4745-bf0e-15706e2f50a6', metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='32bd864c-3920-45ee-8d72-93182b00a70b', metadata={'page': 38.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25')]

In [1]:
import ollama
from IPython.display import Markdown, display
prompt="What is pandas dataframe?" 
response= ollama.chat(model='llama3.2', messages=[{'role':'user','content':prompt}])
response_content = response['message']['content']
display(Markdown(f"**LLM Response:**\n\n{response_content}"))

**LLM Response:**

A pandas DataFrame is a two-dimensional labeled data structure with columns of potentially different types. It is similar to an Excel spreadsheet or a table in a relational database.

DataFrame is a core data structure in the pandas library, which is widely used for data manipulation and analysis in Python. A DataFrame consists of rows and columns, where each column represents a variable or feature, and each row represents an observation or entry.

Here are some key features of a pandas DataFrame:

1. **Labeled axes**: Both rows and columns have labels (index and column name) that can be used to identify them.
2. **Hierarchical index**: DataFrames can have hierarchical indices, where the inner index is subset of the outer index.
3. **Data types**: Each column has a data type (e.g., int64, float64, object), which determines how the data is stored and manipulated.
4. **Data alignment**: When performing operations on DataFrames, pandas aligns the data based on the row and column indices.
5. **Handling missing values**: Pandas provides various ways to handle missing values, including dropping or filling them.

Some common operations performed on DataFrames include:

* Selecting specific columns or rows
* Filtering data using conditions (e.g., selecting rows where a value is greater than 10)
* Grouping and aggregating data
* Merging and joining DataFrames
* Sorting and pivoting data

DataFrames are useful for various applications, such as:

* Data analysis and visualization
* Machine learning and data science
* Business intelligence and reporting
* Scientific computing and research

Here's an example of creating a simple DataFrame:
```python
import pandas as pd

# Create a dictionary with data
data = {'Name': ['John', 'Anna', 'Peter'],
        'Age': [28, 24, 35],
        'Country': ['USA', 'UK', 'Australia']}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

print(df)
```
Output:
```
     Name  Age    Country
0   John   28       USA
1   Anna   24         UK
2  Peter   35  Australia
```

In [13]:
import ollama
from IPython.display import display,Markdown,clear_output
prompt = "what is Acne?"
response_stream= ollama.chat(
    model='llama3.2',
    messages = [{'role':'user', 'content': prompt}],
    stream=True  
)

streamed_response = ""
for token in response_stream:
    streamed_response += token['message']['content']
    clear_output(wait=True)
    display(Markdown(f"**LLM Response (Streaming ):**\n\n{streamed_response}"))


**LLM Response (Streaming ):**

Acne is a skin condition characterized by the occurrence of comedones (blackheads and whiteheads), pimples, cysts, and other types of lesions on the skin. It is a common problem that affects people of all ages, but it is most prevalent during puberty.

There are several factors that contribute to acne, including:

1. **Hormonal fluctuations**: Androgens, such as testosterone, can stimulate oil production in the skin, leading to clogged pores and inflammation.
2. **Clogged pores**: Dead skin cells, oil, and bacteria can combine to form a blockage in the pore, which can lead to acne.
3. **Bacteria**: A type of bacteria called Propionibacterium acnes (P. acnes) is naturally found on the skin and can contribute to acne by producing inflammation and clogged pores.
4. **Genetics**: If your parents had acne, you are more likely to develop it as well.
5. **Environmental factors**: Exposure to certain environmental toxins, such as pollution and stress, can also contribute to acne.

There are several types of acne, including:

1. **Whiteheads**: Small, white bumps on the skin that form when pores become clogged with dead skin cells and oil.
2. **Blackheads**: Dark spots on the skin that form when pores become clogged with dead skin cells, oil, and bacteria.
3. **Papules**: Small, pink bumps on the skin that form when pores become inflamed.
4. **Pustules**: Red, pus-filled bumps on the skin that form when pores are infected with bacteria.
5. **Cysts**: Large, painful bumps on the skin that form when pores become severely inflamed and infected.

Acne can be treated with a variety of methods, including:

1. **Topical treatments**: Creams, gels, or ointments applied directly to the affected area to help reduce inflammation and kill bacteria.
2. **Oral antibiotics**: Antibiotics taken by mouth to help kill bacteria and reduce inflammation.
3. **Hormonal treatments**: Birth control pills or spironolactone may be prescribed to help regulate hormonal imbalances that contribute to acne.
4. **Blue light therapy**: A non-invasive treatment that uses blue light to kill bacteria and reduce inflammation.
5. **Laser treatments**: Ablative and non-ablative laser treatments can help reduce inflammation, kill bacteria, and promote collagen production.

If you are experiencing acne, it is essential to consult with a dermatologist or healthcare professional for proper diagnosis and treatment.

'text'
